In [38]:
import pickle
import importlib
from library import data_preprocess as dp
importlib.reload(dp)
import random
from time import time

import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM, Bidirectional
from keras.layers import Embedding, TimeDistributed, Flatten, Merge, Concatenate
from keras import regularizers
from keras.metrics import sparse_categorical_accuracy, sparse_categorical_crossentropy
from keras.models import load_model
from keras.optimizers import Adam
from keras.models import Model

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import tensorflow as tf
from keras import backend as K

### Variables

In [20]:
# http://nlp.stanford.edu/data/glove.6B.zip
DATA_PATH = './datasets/jokes.pickle'
VOCAB_PATH = './datasets/jokes_vocabulary.pickle'
MODELS_PATH = './models/'

GLOVE_PATH = './data/glove.6B.200d.txt'

MODEL_PREFIX = 'jokes_stacked_lstm_glove'

MAX_SEQUENCE_LENGTH = 13
VALIDATION_SPLIT = 0.2

GLOVE_EMBEDDING_DIM = 200
EMBEDDING_DIM1 = 256
EMBEDDING_DIM2 = 256
HIDDEN_DIM1 = 512
HIDDEN_DIM2 = 256
DEEPER_DIM = 256
DROPOUT_FACTOR = 0.2
REGULARIZATION = 0.00001
LEARNING_RATE = 0.003

DATA_PERCENT = 0.1

RUN_INDEX = 1

In [21]:
with open(DATA_PATH, 'rb') as pickleFile:
    sentences = pickle.load(pickleFile)

with open(VOCAB_PATH, 'rb') as pickleFile:
    vocab = pickle.load(pickleFile)
    
random.shuffle(sentences)

print("Number of sentences = ", len(sentences))
print(sentences[:2])
print("Vocab size = ", len(vocab))
print(vocab[:10])

Number of sentences =  96910
['sos what did the farmer say when he lost his tractor ? have you seen my tractor ? eos', "sos how do find the blind man at the nudist colony ? it's not hard . eos"]
Vocab size =  8922
['sos', 'did', 'you', 'hear', 'about', 'the', 'new', 'corduroy', 'pillows', '?']


In [22]:
# tokenize data
num_words = len(vocab)

tokenizer = Tokenizer(num_words=None, filters='', lower=True, split=' ', 
                      char_level=False, oov_token=None)
tokenizer.fit_on_texts(sentences)
assert num_words == len(tokenizer.word_index)

encoded_sentences = tokenizer.texts_to_sequences(sentences)
print(encoded_sentences[:5])

VOCAB_SIZE = len(tokenizer.word_index) + 1
print(VOCAB_SIZE)

[[1, 17, 36, 5, 963, 60, 34, 31, 317, 53, 1809, 6, 27, 9, 386, 13, 1809, 6, 2], [1, 32, 20, 186, 5, 298, 73, 49, 5, 2202, 3236, 6, 57, 58, 196, 3, 2], [1, 102, 4, 138, 59, 88, 9, 4, 2747, 38, 9, 39, 4, 3335, 3, 2], [1, 7, 232, 8, 277, 291, 5950, 41, 7, 84, 8, 182, 34, 13, 372, 3701, 2], [1, 1117, 23, 96, 8, 118, 4, 1630, 1263, 6, 4928, 23, 646, 118, 17, 9, 77, 1376, 3, 1117, 23, 77, 35, 622, 21, 4928, 23, 9, 67, 7, 221, 28, 419, 3, 2]]
8923


In [23]:
# saving
with open(MODELS_PATH + MODEL_PREFIX + '_tokenizer_' + str(RUN_INDEX) + '.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Preparing Training Data

In [24]:
X_data = []
y_data = []
for sentence in encoded_sentences:
    l = len(sentence)
    sliding_window_length = min(l-3, MAX_SEQUENCE_LENGTH)
    step_size = 1
    for i in range(0, l - sliding_window_length, step_size):
        X_data.append(sentence[i:i+sliding_window_length])
        y_data.append(sentence[i+1:i+sliding_window_length+1])
        
print("Total training data size = ", len(X_data))
MAX_SEQ_LEN = max([len(seq) for seq in X_data])
print("Max seq len = ", MAX_SEQ_LEN)
X_data = pad_sequences(X_data, maxlen=MAX_SEQ_LEN, padding='pre')
y_data = pad_sequences(y_data, maxlen=MAX_SEQ_LEN, padding='pre').reshape(-1, MAX_SEQ_LEN, 1)
#y_data = np.array(y_data).reshape(-1,1)
print(X_data.shape)
print(X_data[:2])
print(y_data.shape)
print(y_data[:2])

Total training data size =  833131
Max seq len =  13
(833131, 13)
[[   1   17   36    5  963   60   34   31  317   53 1809    6   27]
 [  17   36    5  963   60   34   31  317   53 1809    6   27    9]]
(833131, 13, 1)
[[[  17]
  [  36]
  [   5]
  [ 963]
  [  60]
  [  34]
  [  31]
  [ 317]
  [  53]
  [1809]
  [   6]
  [  27]
  [   9]]

 [[  36]
  [   5]
  [ 963]
  [  60]
  [  34]
  [  31]
  [ 317]
  [  53]
  [1809]
  [   6]
  [  27]
  [   9]
  [ 386]]]


In [25]:
print('Indexing glove word vectors')
#Glove Vectors
glove_embeddings_index = {}
f = open(GLOVE_PATH)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embeddings_index[word] = coefs
f.close()
print('Total %s word vectors.' % len(glove_embeddings_index))

Indexing glove word vectors
Total 400000 word vectors.


In [26]:
print('Preparing glove embedding matrix')
glove_embedding_matrix = np.zeros((VOCAB_SIZE, GLOVE_EMBEDDING_DIM))
for word,i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        glove_embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))
print(glove_embedding_matrix.shape)

Preparing glove embedding matrix
Null word embeddings: 255
(8923, 200)


In [45]:
# define model
def StackedLSTM(vocab_size, glove_embedding_dim, glove_embedding_matrix, embedding_dim1, embedding_dim2,
           hidden_dim1, hidden_dim2, deeper_dim, max_seq_len, 
           dropout_factor=0.5, regularization=0.00001, learning_rate=0.001):
    
    inputs = Input(shape=(None,))
    
    glove_embedding = Embedding(vocab_size, glove_embedding_dim, #input_length=max_seq_len,
                                  weights=[glove_embedding_matrix],
                                  mask_zero=True,trainable=False)(inputs)
    
    
    word_embedding = Embedding(vocab_size, embedding_dim1, #input_length=max_seq_len, 
                               mask_zero=True, embeddings_regularizer=regularizers.l2(regularization))(inputs)
    
    concat_embeds = Concatenate(axis=-1)([glove_embedding, word_embedding])
    
    final_embed = Dense(units=embedding_dim2, activation='tanh',
                        kernel_regularizer=regularizers.l2(regularization))(concat_embeds)
    
    lstm1 = LSTM(hidden_dim1, activation='tanh', 
                   kernel_regularizer=regularizers.l2(regularization), 
                   recurrent_regularizer=regularizers.l2(regularization), #unroll=True, 
                   return_sequences = True, dropout=dropout_factor, recurrent_dropout=dropout_factor)(final_embed)
    
    lstm2 = LSTM(hidden_dim2, activation='tanh', 
                   kernel_regularizer=regularizers.l2(regularization), 
                   recurrent_regularizer=regularizers.l2(regularization), #unroll=True, 
                   return_sequences = True, dropout=dropout_factor, recurrent_dropout=dropout_factor)(lstm1)
    
    timedist_dropout = TimeDistributed(Dropout(dropout_factor))(lstm2)
    
    deep_dense = Dense(units=deeper_dim, activation='tanh', 
                       kernel_regularizer=regularizers.l2(regularization))(timedist_dropout)
    
    dropout_layer1 = Dropout(dropout_factor)(deep_dense)
    
    outputs = Dense(units=vocab_size, activation='softmax', 
                    kernel_regularizer=regularizers.l2(regularization))(dropout_layer1)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=learning_rate),
                  metrics=[sparse_categorical_crossentropy, sparse_categorical_accuracy], sample_weight_mode='temporal')
    return model

In [46]:
K.clear_session()
sess = tf.Session()
K.set_session(sess)

model = StackedLSTM(vocab_size=VOCAB_SIZE, glove_embedding_dim=GLOVE_EMBEDDING_DIM,
                    glove_embedding_matrix=glove_embedding_matrix, 
                    embedding_dim1=EMBEDDING_DIM1, embedding_dim2=EMBEDDING_DIM2,
                    hidden_dim1=HIDDEN_DIM1, hidden_dim2=HIDDEN_DIM2,
                    deeper_dim=DEEPER_DIM, max_seq_len=MAX_SEQ_LEN, dropout_factor=DROPOUT_FACTOR, 
                    regularization=REGULARIZATION, learning_rate=LEARNING_RATE)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    1784600     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    2284288     input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 456)    0           embedding_1[0][0]                
                                                                 embedding_2[0][0]                
__________

In [47]:
class TB(TensorBoard):
    def __init__(self, log_every=1, **kwargs):
        super().__init__(**kwargs)
        self.log_every = log_every
        self.counter = 0
    
    def on_batch_end(self, batch, logs=None):
        self.counter+=1
        if self.counter%self.log_every==0:
            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, self.counter)
            self.writer.flush()
        
        super().on_batch_end(batch, logs)

In [48]:
start_time = time()
tensorboard = TB(log_dir="./logs/" + MODEL_PREFIX + "/{}".format(time()), 
                          histogram_freq=0, write_graph=True, write_images=False, log_every=10)

callbacks=[tensorboard, 
           EarlyStopping(patience=5, monitor='val_loss'),
           ModelCheckpoint(filepath=MODELS_PATH + 'checkpoints/'+ MODEL_PREFIX + '_gen' + str(RUN_INDEX) + '.{epoch:02d}-{val_loss:.2f}.hdf5', 
                           monitor='val_loss', verbose=1, mode='auto', period=1), 
           ModelCheckpoint(filepath=MODELS_PATH + MODEL_PREFIX + '_gen'+str(RUN_INDEX)+'.hdf5', 
                           monitor='val_loss', verbose=1, mode='auto', period=1, save_best_only=True)]

model.fit(X_data, y_data, epochs=20, batch_size=2048, shuffle=True, verbose=1, validation_split=0.2, callbacks=callbacks)

print("Total elapsed time: ", time()-start_time)

Train on 666504 samples, validate on 166627 samples
Epoch 1/20
666504/666504 [==============================] - 473s 710us/step - loss: 6.4779 - sparse_categorical_crossentropy: 6.4545 - sparse_categorical_accuracy: 0.0370 - val_loss: 6.2857 - val_sparse_categorical_crossentropy: 6.2554 - val_sparse_categorical_accuracy: 0.0396

Epoch 00001: saving model to ./models/checkpoints/jokes_stacked_lstm_glove_gen1.01-6.29.hdf5

Epoch 00001: val_loss improved from inf to 6.28574, saving model to ./models/jokes_stacked_lstm_glove_gen1.hdf5
Epoch 2/20
666504/666504 [==============================] - 476s 714us/step - loss: 5.4573 - sparse_categorical_crossentropy: 5.3864 - sparse_categorical_accuracy: 0.1266 - val_loss: 4.8011 - val_sparse_categorical_crossentropy: 4.6716 - val_sparse_categorical_accuracy: 0.2131

Epoch 00002: saving model to ./models/checkpoints/jokes_stacked_lstm_glove_gen1.02-4.80.hdf5

Epoch 00002: val_loss improved from 6.28574 to 4.80107, saving model to ./models/jokes_sta

KeyboardInterrupt: 

In [86]:
# generate a sequence from a language model
def generate(model, tokenizer, seed_text, maxlen, probabilistic=False, exploration_factor=1.0):
    
    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
    seq = tokenizer.texts_to_sequences([seed_text])[0]
    print(seq)
    
    while True:
        encoded_seq = seq
        if len(seq) > MAX_SEQ_LEN:
            encoded_seq = encoded_seq[-1*MAX_SEQ_LEN:]
            
        #padded_seq = pad_sequences([encoded_seq], maxlen=MAX_SEQ_LEN, padding='pre')
        padded_seq = np.array([seq])
        y_prob = model.predict(padded_seq)[0][-1].reshape(1,-1)#[3:].reshape(-1,1)
        
        if random.random() <= exploration_factor:
            probabilistic = True
        else:
            probabilistic = False
            
        if probabilistic:
            y_class = np.argmax(np.random.multinomial(1,y_prob[0]/(np.sum(y_prob[0])+1e-5),1))
        else:
            y_class = y_prob.argmax(axis=-1)[0]
        
        if y_class == 0:
            break
        out_word = reverse_word_map[y_class]
        seq.append(y_class)
        if out_word == 'eos' or len(seq) > maxlen or out_word == 'sos':
            break
    
    words = [reverse_word_map[idx] for idx in seq]
    
    return ' '.join(words)

In [87]:
joke = generate(model, tokenizer, "sos i had to use", maxlen=40)
print(joke)

[1, 7, 84, 8, 258]
sos i had to use to play software aids . so i learned who's started to run it up in the bus . eos


In [88]:
def bigrams_list(sentence):
    words = sentence.split(' ')
    bigrams = []
    for i in range(0, len(words)-1):
        bigrams.append(words[i]+' '+words[i+1])
    return bigrams

print(bigrams_list("sos hello , i'm a dinosaur . eos"))

['sos hello', 'hello ,', ", i'm", "i'm a", 'a dinosaur', 'dinosaur .', '. eos']


In [89]:
sentence_bigrams = [bigrams_list(s) for s in sentences]
print(sentence_bigrams[:2])

[['sos what', 'what did', 'did the', 'the farmer', 'farmer say', 'say when', 'when he', 'he lost', 'lost his', 'his tractor', 'tractor ?', '? have', 'have you', 'you seen', 'seen my', 'my tractor', 'tractor ?', '? eos'], ['sos how', 'how do', 'do find', 'find the', 'the blind', 'blind man', 'man at', 'at the', 'the nudist', 'nudist colony', 'colony ?', "? it's", "it's not", 'not hard', 'hard .', '. eos']]


In [90]:
def intersection(lst1, lst2):
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

def similarity_score(lst1, lst2):
    intersection_len = len(intersection(lst1, lst2))
    return (1.0*intersection_len)/len(lst1)#+len(lst2)-intersection_len)
 
def print_closest_sentences(sentence, sentence_bigrams, top_k=3):
    bigrams = bigrams_list(sentence)
    scores = np.array([similarity_score(bigrams, sbigrams)
                       for sbigrams in sentence_bigrams])
    top_k_indices = scores.argsort()[-1*top_k:][::-1]
    top_k_scores = scores[top_k_indices]
    for k in range(top_k):
        print(top_k_scores[k], " -> ", sentences[top_k_indices[k]])

In [91]:
print_closest_sentences(joke, sentence_bigrams, 10)

0.2608695652173913  ->  sos i couldn't get tickets for the plan b concert . so i had to go with my first choice instead . eos
0.2608695652173913  ->  sos ran out of toilet paper , so i had to use leaves . just kidding , but my son learned a big lesson about leaving his clothes in the bathroom . eos
0.2608695652173913  ->  sos my wife was out of town , so i had to run the morning routine by myself today . i learned a lot . for example , apparently i have two kids . eos
0.2608695652173913  ->  sos ran out of toilet paper , so i had to use leaves . just kidding , but my son learned a big lesson about leaving his clothes in the bathroom . eos
0.2608695652173913  ->  sos i wanted to have sex with my girlfriend , but she was on her period . so i had to pull some strings . eos
0.21739130434782608  ->  sos a woman stopped me in the street and asked me to show her how to get to the hospital . so i pushed her under a bus . eos
0.21739130434782608  ->  sos guinness book of world records i use to 

In [92]:
joke = generate(model, tokenizer, "sos what do you call", maxlen=40)
print(joke)
print_closest_sentences(joke, sentence_bigrams, 10)

[1, 17, 20, 9, 66]
sos what do you call a pilot threedots try to have sex , mary . eos
0.5333333333333333  ->  sos what do you call a man who expects to have sex on the second date ? patient . eos
0.5333333333333333  ->  sos what do you call a black guy flying a plane ? a pilot threedots you racist bastard . eos
0.5333333333333333  ->  sos what do you call a man who expects to have sex on the second date ? slow . eos
0.4666666666666667  ->  sos what do you call a black guy flying a plane ? a pilot . eos
0.4666666666666667  ->  sos what do you call a muslim flying a plane ? a pilot you racists . eos
0.4666666666666667  ->  sos what do you call a black man flying an airplane ? a pilot , you fucking racist . eos
0.4666666666666667  ->  sos what do you call a black man that flies planes ? a pilot , you racist . eos
0.4666666666666667  ->  sos what do you call a black guy flying a plane ? a pilot you racist bastard . eos
0.4666666666666667  ->  sos what do you call a black man flying a plan

In [93]:
print(sentences[:10])

['sos what did the farmer say when he lost his tractor ? have you seen my tractor ? eos', "sos how do find the blind man at the nudist colony ? it's not hard . eos", 'sos only a woman can make you a millionaire if you are a billionaire . eos', 'sos i used to play water polo but i had to stop when my horse drowned eos', "sos harry : want to see a magic trick ? voldemort : let's see what you got potter . harry : got your nose ! voldemort : you know i hate that game . eos", "sos what's the difference between michael phelps and hitler threedots michael phelps can finish a race eos", 'sos * tries to get a life . life : i have a girlfriend . eos', "sos it funny that when it's black on white , it's a crime . when it it's white on black , it's a hate crime . eos", "sos living in switzerland wouldn't be so bad . the flag is a plus . eos", 'sos yo mama is so hairy that bigfoot tried to take her picture ! eos']


In [94]:
joke = generate(model, tokenizer, "sos what's the difference between being", maxlen=40)
print(joke)
print_closest_sentences(joke, sentence_bigrams, 10)

[1, 56, 5, 76, 74, 167]
sos what's the difference between being proud , gay man hides in switzerland ? threedots it's just that spicy . eos
0.3  ->  sos what's the difference between a gay man and a refrigerator ? my refrigerator doesn't fart when i pull my meat out of it . eos
0.3  ->  sos what's the difference between a gay man and a freezer ? the freezer doesn't fart when you pull the meat out . eos
0.3  ->  sos what's the difference between a gay man and a freezer ? the freezer doesn't fart when you pull your meat out . eos
0.3  ->  sos what's the difference between being hungry and horny ? where you put the cucumber . eos
0.3  ->  sos what's the difference between a sliced up body and a new bmw ? threedots i don't have a new bmw in my garage . eos
0.3  ->  sos what's the difference between a terrorist cell and a children's hospital ? threedots don't ask me man , i just fly the drones . eos
0.3  ->  sos what's the difference between a ferrari and a boner ? threedots i have a boner 

In [29]:
model1 =  load_model('models/checkpoints/jokes_bilstm_gen2.08-4.39.hdf5')

In [100]:
joke = generate(model, tokenizer, "sos i am a boy who likes", maxlen=40)
print(joke)
print_closest_sentences(joke, sentence_bigrams, 10)

[1, 7, 237, 4, 287, 71, 720]
sos i am a boy who likes me , there's a better place to drink nice stuff until that i sleep with a bunch of slippers when she giggles . eos
0.16666666666666666  ->  sos i sleep with a knife under my bed in case i can't open my midnight snacks . it also comes in handy if people try to steal them . eos
0.16666666666666666  ->  sos i think i can speak for everyone when i say that i am a ventriloquist . eos
0.16666666666666666  ->  sos i sleep with a water gun near my bed , in case of cat burglar . eos
0.16666666666666666  ->  sos i sleep with a squirt gun under my pillow just in case a gang of cats break in while i'm sleeping . eos
0.16666666666666666  ->  sos i treat my body like a temple . by that i mean that a bunch of jewish guys enter me every friday night . eos
0.16666666666666666  ->  sos a lady that sat next to me on a plane freaked out when she realized that i am a muslim . i laughed so hard my grenades nearly fell out of my pockets . eos
0.1666666666